### Part 1: SQL-Based Table Comparison

1. Import these two tables as SQL tables using any SQL-related Python library.


In [1]:
import pandas as pd
from pandasql import sqldf

df1 = pd.read_csv("products_day1.csv")
df2 = pd.read_csv("products_day2.csv")
print(df1.shape)
print(df2.shape)

# Defining query function
pysqldf = lambda q: sqldf(q, globals())

(6, 5)
(6, 5)


In [2]:
df1.head() # checking top 5 rows in table 1

,product_id,name,category,price,stock
0,101,Wireless Mouse,Electronics,25.99,120
1,102,Water Bottle,Kitchen,10.50,200
2,103,Bluetooth Speaker,Electronics,49.99,75
3,104,Yoga Mat,Fitness,20.00,60
4,105,Desk Lamp,Furniture,35.00,90


In [3]:
df2.head() # checking top 5 rows in table 2

,product_id,name,category,price,stock
0,101,Wireless Mouse,Electronics,23.99,120
1,102,Water Bottle,Kitchen,10.50,180
2,103,Bluetooth Speaker,Electronics,49.99,75
3,104,Yoga Mat,Fitness,20.00,60
4,105,Desk Lamp,Furniture,37.00,90


2. Write SQL queries to:
- Identify full rows that were ADDED or REMOVED.

In [4]:
query = """
SELECT *, "removed" as change_type
from df1 where df1.product_id not in (select distinct product_id from df2)
UNION
SELECT *, "added" as change_type
from df2 where df2.product_id not in (select distinct product_id from df1)

"""

sql_task_2 = pysqldf(query)
sql_task_2

,product_id,name,category,price,stock,change_type
0,106,Notebook,Stationery,2.99,300,removed
1,107,Smart Watch,Wearables,99.00,50,added


- Compare matching rows and detect column-level changes (e.g., price, stock).

In [5]:
query = """
SELECT 
    df1.product_id,
    'price' AS column_changed,
    df1.price AS old_value,
    df2.price AS new_value
FROM df1
JOIN df2 ON df1.product_id = df2.product_id
WHERE df1.price != df2.price

UNION ALL

SELECT 
    df1.product_id,
    'stock' AS column_changed,
    df1.stock AS old_value,
    df2.stock AS new_value
FROM df1
JOIN df2 ON df1.product_id = df2.product_id
WHERE df1.stock != df2.stock
order by df1.product_id;

"""
sql_task_3 = pysqldf(query)
sql_task_3

,product_id,column_changed,old_value,new_value
0,101,price,25.99,23.99
1,102,stock,200.00,180.00
2,105,price,35.00,37.00
